In [44]:
import torch
from trainer.Model1 import UNet

import os

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch import optim
from torch.optim import lr_scheduler
from torchvision import transforms

import numpy as np
import pandas as pd
from PIL import Image
import time

import librosa
import librosa.display

import matplotlib.pyplot as plt

import deepdish as dd
from multiprocessing import Pool, set_start_method

import utils

In [2]:
import warnings
warnings.simplefilter('ignore')

In [3]:
AUDIO_DIR = 'data/fma_small'
tracks = utils.load('data/fma_metadata/tracks.csv')
genres = utils.load('data/fma_metadata/genres.csv')
features = utils.load('data/fma_metadata/features.csv')
echonest = utils.load('data/fma_metadata/echonest.csv')

In [5]:
small_tracks = tracks[tracks['set', 'subset'] <= 'small']
small_genres = ['Electronic', 'Experimental', 'Folk', 'Hip-Hop', 'Instrumental', 'International', 'Pop', 'Rock']

# Load preprocessed files

In [36]:
X = dd.io.load('data/fma_preprocessed/X.h5')
Y = dd.io.load('data/fma_preprocessed/Y.h5')

In [37]:
# Assuming files are preprocessed as image (256x256x3), labels are just strings
n = 3
print(f'{"X":>31s}{"Y":>20s}')
print(f"Training:   {str(X['training'].shape):>20s}{str(Y['training'].shape):>20s}")
print(f"Validation: {str(X['validation'].shape):>20s}{str(Y['validation'].shape):>20s}")
print(f"Test:       {str(X['test'].shape):>20s}{str(Y['test'].shape):>20s}")

                              X                   Y
Training:   (19170, 1, 256, 256)          (19170, 8)
Validation:  (2399, 1, 256, 256)           (2399, 8)
Test:        (2399, 1, 256, 256)           (2399, 8)


## Define Dataset and Model classes

In [67]:
class FMADataset(Dataset):
    def __init__(self, mode='training'):
        if mode not in ('training', 'validation', 'test'):
            raise Error('mode must be training, validation, or test.')
        self.mode = mode
        self.x = X[mode]
        self.y = Y[mode]
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        # ndarray contains negative strides
        # to fix, we flip a copy, and flip again
        image = torch.from_numpy(np.flip(self.x[index], axis=0).astype('float64').copy())
        image = torch.flip(image, dims=(0,))
        image = transforms.Normalize(256, 256)(image)
        label = self.y[index]
        return image.float(), label

In [122]:
class Net(nn.Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = nn.Sequential(
          # Defining a 2D convolution layer
          nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(64),
          nn.ReLU(inplace=True),
          nn.MaxPool2d(kernel_size=2, stride=2),
          # Defining another 2D convolution layer
          nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(64),
          nn.ReLU(inplace=True),
          nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.out = nn.Sequential(
          nn.Linear(64 * 64 * 64, 8),
          nn.Softmax()
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.out(x)
        return x

In [110]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


## First overfit on a single batch

Current model does not seem to be learning

In [ ]:
# Use dotdict util to emulate argparse
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

In [123]:
train_set = FMADataset(mode='training')
val_set = FMADataset(mode='validation')
train_loader = DataLoader(train_set, batch_size=args.batch_size, shuffle=False)
val_loader = DataLoader(val_set, batch_size=args.batch_size, shuffle=False)
model = Net()
optim = torch.optim.Adam(model.parameters(), lr=args.learning_rate)
# scheduler = None
scheduler = lr_scheduler.OneCycleLR(
    optim, 
    max_lr=args.learning_rate, 
    epochs=args.num_epochs,
    steps_per_epoch=len(train_loader)
)
loss_function = torch.nn.CrossEntropyLoss()

args = {
    'learning_rate': 0.01,
    'batch_size': 16,
    'num_epochs': 100,
    'job_directory': 'saved_models'
}

args = dotdict(args)

for ep in range(args.num_epochs):
    xs, ys = next(iter(train_loader))
    xs, ys = xs.to(device), ys.to(device) 
    optim.zero_grad()
    probs = model(xs)
    ys = torch.max(ys, 1)[1] # Turns out nn.Softmax() takes ints (0-7), not 1-hot encoded vectors, oops.
    loss = loss_function(probs, ys)    
    loss.backward()
    optim.step()
    if scheduler is not None:
        scheduler.step()

    print(f'Epoch {ep} Loss {loss.item()}')

Epoch 0 Loss 2.029628038406372
Epoch 1 Loss 1.4625848531723022
Epoch 2 Loss 1.461548089981079
Epoch 3 Loss 1.4615304470062256
Epoch 4 Loss 1.4615230560302734
Epoch 5 Loss 1.4615168571472168
Epoch 6 Loss 1.4615130424499512
Epoch 7 Loss 1.4615108966827393
Epoch 8 Loss 1.4615099430084229
Epoch 9 Loss 1.4615094661712646
Epoch 10 Loss 1.4615092277526855
Epoch 11 Loss 1.4615089893341064
Epoch 12 Loss 1.4615089893341064
Epoch 13 Loss 1.4615089893341064
Epoch 14 Loss 1.4615089893341064
Epoch 15 Loss 1.4615089893341064
Epoch 16 Loss 1.4615089893341064
Epoch 17 Loss 1.4615089893341064
Epoch 18 Loss 1.4615089893341064
Epoch 19 Loss 1.4615089893341064
Epoch 20 Loss 1.4615089893341064
Epoch 21 Loss 1.4615089893341064
Epoch 22 Loss 1.4615089893341064
Epoch 23 Loss 1.4615089893341064
Epoch 24 Loss 1.4615089893341064
Epoch 25 Loss 1.4615089893341064
Epoch 26 Loss 1.4615089893341064
Epoch 27 Loss 1.4615089893341064
Epoch 28 Loss 1.4615089893341064
Epoch 29 Loss 1.4615089893341064
Epoch 30 Loss 1.461508

In [103]:
   
args = {
    'learning_rate': 0.001,
    'batch_size': 300,
    'num_epochs': 4,
    'job_directory': 'saved_models'
}

args = dotdict(args)

In [114]:
def fit(args):
    train_set = FMADataset(mode='training')
    val_set = FMADataset(mode='validation')
    
    train_loader = DataLoader(train_set, batch_size=args.batch_size, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=args.batch_size, shuffle=False)
    
    model = Net()
    optim = torch.optim.Adam(model.parameters(), lr=args.learning_rate)
    # scheduler = None
    scheduler = lr_scheduler.OneCycleLR(
        optim, 
        max_lr=args.learning_rate, 
        epochs=args.num_epochs,
        steps_per_epoch=len(train_loader)
    )
    loss_function = torch.nn.NLLLoss()
    
    # Keep track of metrics
    total_train_loss, total_val_loss = [], []
    
    print('Train Loss    Val Loss')
    for epoch in range(args.num_epochs):
        start = time.time() # Track elapsed time for training/validation
        model.train()
        for xs, ys in train_loader:
            xs, ys = xs.to(device), ys.to(device)
            
            optim.zero_grad()
            probs = model(xs)
            ys = torch.max(ys, 1)[1]
            loss = loss_function(probs, ys)
            
            loss.backward()
            optimizer.step()
            if scheduler is not None:
                scheduler.step()
            
            # Track metrics
            total_train_loss.append(loss.item())
            
        model.eval()
        with torch.no_grad():
            for xs, ys in val_loader:
                xs, ys = xs.to(device), ys.to(device)
                probs = model(xs)
                loss = loss_function(probs, ys)
                
                # Track metrics
                total_val_loss.append(loss.item())
                                      
        elapsed = time.time() - start
        print(f'{epoch:5d}{total_train_loss[-1]:14.4f}{total_val_loss[-1]:12.4f}')
        timer(elapsed)
    
    export_path = os.path.join(args.job_dir, 'test_model.pth')
    torch.save(model.state_dict(), export_path)

In [1]:
train_set = FMADataset(mode='training')
train_loader = DataLoader(train_set, batch_size=args.batch_size, shuffle=True)
xs, ys = next(iter(train_loader))
print(xs.shape, ys.shape)
probs.shape

NameError: name 'FMADataset' is not defined